# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 8.81MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [2]:
!pip install langchain-huggingface langchain langchain-community langchain-chroma Chroma langchain-groq

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [90]:
import pandas as pd
df = pd.read_csv("/content/Dataset.csv")
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [91]:
df.shape

(104, 2)

In [92]:
!pip install pyarabic

In [93]:
from pyarabic.araby import strip_harakat, strip_tashkeel , strip_lastharaka

In [94]:
def preprocess(text):
  text = strip_tashkeel(text)
  text = strip_harakat(text)
  text = strip_lastharaka(text)
  return text


df['المخالفة'] = df['المخالفة'].apply(preprocess)
df['الغرامة'] = df['الغرامة'].apply(preprocess)
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [95]:
markdown_files = []

for i in range(0,104):
  title = df['المخالفة'].iloc[i]
  content = df['الغرامة'].iloc[i]

  markdown_content = f"**{title}** {content}"
  markdown_files.append(markdown_content)

In [96]:
markdown_files

['**قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.** الغرامة المالية 100 - 150 ريال',
 '**ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.** الغرامة المالية 100 - 150 ريال',
 '**عدم وجود تأمين ساري للمركبة.** الغرامة المالية 100 - 150 ريال',
 '**عبور المشاة للطرق من غير الأماكن المخصصة لهم.** الغرامة المالية 100 - 150 ريال',
 '**عدم تقيد المشاة بالإشارات الخاصة بهم.** الغرامة المالية 100 - 150 ريال',
 '**وقوف المركبة في أماكن غير مخصصة للوقوف.** الغرامة المالية 100 - 150 ريال',
 '**عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم.** الغرامة المالية 100 - 150 ريال',
 '**عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار.** الغرامة المالية 150 - 300 ريال',
 '**الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين مترا.** الغرامة المالية 150 - 300 ريال',
 '**قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر.** الغرامة المالية 150 - 300 ريال',
 '**مخالفة قواع

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [97]:
!pip install -qU langchain-text-splitters

In [151]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
chunks = text_splitter.create_documents(markdown_files)

In [152]:
chunks

[Document(metadata={}, page_content='**قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.** الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.** الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**عدم وجود تأمين ساري للمركبة.** الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**عبور المشاة للطرق من غير الأماكن المخصصة لهم.** الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**عدم تقيد المشاة بالإشارات الخاصة بهم.** الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**وقوف المركبة في أماكن غير مخصصة للوقوف.** الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم.** الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار.** الغرامة المالية 150 - 300 ر

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [153]:
from langchain_community.embeddings import SentenceTransformerEmbeddings

In [154]:
embedding_function = SentenceTransformerEmbeddings(model_name="Omartificial-Intelligence-Space/GATE-AraBert-v0")

In [155]:
from langchain_chroma import Chroma

In [156]:
vector_store =  Chroma.from_documents(chunks, embedding_function, persist_directory="./chroma_db")

In [157]:
persist_directory="./chroma_db"
db = Chroma(persist_directory=persist_directory, embedding_function=embedding_function)

In [158]:
res1 = db.similarity_search(' عقوبة عدم الوقوف')
res1

[Document(metadata={}, page_content='**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال'),
 Document(metadata={}, page_content='**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال'),
 Document(metadata={}, page_content='**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال'),
 Document(metadata={}, page_content='**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال')]

In [159]:
res2 = db.similarity_search_with_relevance_scores(' عقوبة عدم الوقوف')
res2

<ipython-input-159-bca06c918bc7>:1: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={}, page_content='**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال'), -282.089189942109), (Document(metadata={}, page_content='**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال'), -282.089189942109), (Document(metadata={}, page_content='**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال'), -282.089189942109), (Document(metadata={}, page_content='**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال'), -282.089189942109)]
  res2 = db.similarity_search_with_relevance_sco

[(Document(metadata={}, page_content='**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال'),
  -282.089189942109),
 (Document(metadata={}, page_content='**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال'),
  -282.089189942109),
 (Document(metadata={}, page_content='**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال'),
  -282.089189942109),
 (Document(metadata={}, page_content='**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال'),
  -282.089189942109)]

## Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template greets the user first, states that the information provided could be incorrect, and advises the user to visit the traffic initiative website to verify. Additionally, provide the user with advice in Arabic, ensuring it stays within the given context.

In [185]:
from langchain_core.prompts import PromptTemplate

template = """
رحب بالمستخدم وثم اعطه تعليمات ان المعلومات قد تكون خاطئة واعطه نصيحه
--
context: {context}
--
question: {question}
--
Your Answer:
"""
prompt_template = PromptTemplate(
    template = template,
    input_variables = ['context' , 'question']
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [186]:
from langchain_groq import ChatGroq

In [187]:
model = ChatGroq(temperature=0, groq_api_key="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", model_name="llama3-8b-8192")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [188]:
from langchain.chains import LLMChain

llm_chain = LLMChain(
    llm= model,
    prompt=prompt_template,
    verbose=1
)


## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [189]:
def query_rag(query: str):
  sim_search_res =db.similarity_search_with_score(query , k=4)
  context_text = ''
  for result in sim_search_res:
    context_text += result[0].page_content

  rag_response = llm_chain.invoke({'context': context_text , 'question': query})
  return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [190]:
query = "ماهي عقوبة عدم الوقوف وقوفاً تاماً عند اشارة؟"
query_rag(query)

Prompt after formatting:

رحب بالمستخدم وثم اعطه تعليمات ان المعلومات قد تكون خاطئة واعطه نصيحه 
--
context: **عدم الوقوف تماماً عند إشارة قف.** الغرامة المالية 500 - 900 ريال**عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له الأفضلية.** الغرامة المالية 500 - 900 ريال**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال
--
question: ماهي عقوبة عدم الوقوف وقوفاً تاماً عند اشارة؟
--
Your Answer:


> Finished chain.


{'context': '**عدم الوقوف تماماً عند إشارة قف.** الغرامة المالية 500 - 900 ريال**عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له الأفضلية.** الغرامة المالية 500 - 900 ريال**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال**عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.** الغرامة المالية 3000 - 6000 ريال',
 'question': 'ماهي عقوبة عدم الوقوف وقوفاً تاماً عند اشارة؟',
 'text': 'مرحباً بك! شكراً لزيارتك. قبل أن أبدأ في الإجابة عن سؤالك، أريد أن أشدد على أن المعلومات المُقدمة قد تكون خاطئة أو قديمة، لذا من المهم أن نتحقق من صحة المعلومات قبل اتخاذ أي قرار.\n\nبناءً على المعلومات المُقدمة، عقوبة عدم الوقوف وقوفاً تاماً عند إشارة قف هي الغرامة المالية 500 - 900 ريال.\n\nأيضاً، أريد أن أتمنى لك أن تكون حذراً على الطريق وتبدأ في الوقوف عند الإشارات والمراكز الأمنية عندما يلزم ذلك، 

In [191]:
query = 'ماهي عقوبة سير المركبة بلا لوحة خلفية؟'
res1 = query_rag(query)
res1

Prompt after formatting:

رحب بالمستخدم وثم اعطه تعليمات ان المعلومات قد تكون خاطئة واعطه نصيحه 
--
context: **سير المركبة بلا لوحة أمامية.** الغرامة المالية 1000 - 2000 ريال**سير المركبة بلا لوحة أمامية.** الغرامة المالية 1000 - 2000 ريال**سير المركبة بلا لوحة أمامية.** الغرامة المالية 1000 - 2000 ريال**سير المركبة بلا لوحة أمامية.** الغرامة المالية 1000 - 2000 ريال
--
question: ماهي عقوبة سير المركبة بلا لوحة خلفية؟
--
Your Answer:


> Finished chain.


{'context': '**سير المركبة بلا لوحة أمامية.** الغرامة المالية 1000 - 2000 ريال**سير المركبة بلا لوحة أمامية.** الغرامة المالية 1000 - 2000 ريال**سير المركبة بلا لوحة أمامية.** الغرامة المالية 1000 - 2000 ريال**سير المركبة بلا لوحة أمامية.** الغرامة المالية 1000 - 2000 ريال',
 'question': 'ماهي عقوبة سير المركبة بلا لوحة خلفية؟',
 'text': 'مرحباً بك! شكراً لزيارتك. قبل أن أبدأ في الإجابة عن سؤالك، أريد أن أشدد على أن المعلومات المُقدمة قد تكون خاطئة أو قديمة، لذا من المهم أن نتحقق من صحة المعلومات قبل اتخاذ أي قرار.\n\nبناءً على المعلومات المُقدمة، سير المركبة بلا لوحة أمامية هو المُentioned، وليس لوحة خلفية. الغرامة المالية المُentioned هي 1000-2000 ريال.\n\nلذلك، من المهم أن نتحقق من صحة المعلومات قبل الإجابة عن سؤالك. في هذه الحالة، سير المركبة بلا لوحة أمامية هو المُentioned، وليس لوحة خلفية.\n\nنصيحتي لك هي أن تحقق من صحة المعلومات قبل اتخاذ أي قرار، وخاصة فيما يتعلق بالغرامات المالية والقوانين.'}

In [192]:
query = ' هل هناك عقوبة لترك الأطفال دون سن العاشرة في المركبة دون مرافق راشد؟'
res1 = query_rag(query)
res1

Prompt after formatting:

رحب بالمستخدم وثم اعطه تعليمات ان المعلومات قد تكون خاطئة واعطه نصيحه 
--
context: **قيادة مركبة دون توافر التجهيزات اللازمة، مثل المكابح والأنوار وما في حكمهما.** الغرامة المالية 1000 - 2000 ريال**عدم قيام السائق في حال إغلاق جزء من الطريق بإعطاء الأفضلية لمن كان طريقه مفتوحاً.** الغرامة المالية 500 - 900 ريال**عدم قيام السائق في حال إغلاق جزء من الطريق بإعطاء الأفضلية لمن كان طريقه مفتوحا.** الغرامة المالية 500 - 900 ريال**عدم قيام السائق في حال إغلاق جزء من الطريق بإعطاء الأفضلية لمن كان طريقه مفتوحا.** الغرامة المالية 500 - 900 ريال
--
question:  هل هناك عقوبة لترك الأطفال دون سن العاشرة في المركبة دون مرافق راشد؟
--
Your Answer:


> Finished chain.


{'context': '**قيادة مركبة دون توافر التجهيزات اللازمة، مثل المكابح والأنوار وما في حكمهما.** الغرامة المالية 1000 - 2000 ريال**عدم قيام السائق في حال إغلاق جزء من الطريق بإعطاء الأفضلية لمن كان طريقه مفتوحاً.** الغرامة المالية 500 - 900 ريال**عدم قيام السائق في حال إغلاق جزء من الطريق بإعطاء الأفضلية لمن كان طريقه مفتوحا.** الغرامة المالية 500 - 900 ريال**عدم قيام السائق في حال إغلاق جزء من الطريق بإعطاء الأفضلية لمن كان طريقه مفتوحا.** الغرامة المالية 500 - 900 ريال',
 'question': ' هل هناك عقوبة لترك الأطفال دون سن العاشرة في المركبة دون مرافق راشد؟',
 'text': 'مرحباً بك! أهلاً وسهلاً في هذا المكان. قبل أن أبدأ في الإجابة عن سؤالك، أريد أن أشدد على أن المعلومات التي سأقدمها قد تكون خاطئة، لذلك من المهم أن نكون حذرين في تفسيرها.\n\nبما أنك سألت عن عقوبة ترك الأطفال دون سن العاشرة في المركبة دون مرافق راشد، أريد أن أشير إلى أن هذا النوع من المخالفات يخضع لتعليمات وقيود محددة في قوانين القيادة في المملكة العربية السعودية.\n\nوفقاً لما هو متاح، لا توجد عقوبة مالية محددة لترك الأطفال دون